# pytest

> [Main Table of Contents](../../README.md)

## In This Notebook
- Directory structure
- Run pytest
	- Identify Node ID
- Test structure
- Assert Statements
	- Assert with failure message
- Pytest Methods
- Test for exceptions
- Test Setup and Teardown
	- Fixtures
		- Create fixtures
		- Use fixtures
		- Use multiple fixtures - fixture chaining
- Mocking
- Test Data Science Models
- Test Plots
- What to test
- Pytest report
- Continuous Integration and Codecov

In [29]:
import pytest

## Run pytest
- Command line invocation

- Find pattern in test function names and test class names and only run if matched substring. Useful for large test collections.
	```python
	pytest -k pattern
	```

- Exit immediately on first fail. Useful for large files.
	```python
	pytest -x file_or_dir
	```
- Execute specific part of test file or test class with nodeID
	```python
	pytest <path_with_node_id_notation>
	```
- Show reason xfail
	```python
	pytest -rx file_or_dir
	```
- Show reason for skipped test
	```python
	pytest -rs file_or_dir
	```
- Show reason for both skipped and xfailed test
	```python
	pytest -rs file_or_dir
	```

### Identify Node ID
- Node ID is the last part od the following
	```python
	# Node ID of a test class
	<path_test_module>::<test_class_name>

	# Node ID of unit test
	<path_test_module>::<test_class_name>::<unit_test_name>
	```

## Directory structure
- `tests` directory should mirror source code directory

## Test structure
- Tests for each functions are organized into a class

## Assert statements
- `assert` is main keyword used in unit tests

### Assert with failure message
- Best practice to include message with assert statement which prints on failure

In [30]:
def add(a, b):
    return a+b

class TestAdd():
    def test_integers():
        expected = 9
        actual = add(3, 6)
        # Message should include function as called and actual return value
        message = f'add(3, 6) should return the int 9, but it actually returned {actual}'
        assert actual == expected, message

## pytest methods

Method | Description | Example
--- | --- | ---
pytest.approx(#) | Handle float datatypes | assert 0.1 + 0.1 + 0.1 == pytest.approx(0.3)<br>np.array([0.1 + 0.1, 0.1 + 0.1]) == pytest.approx([0.2, 0.2])
pytest.raises(ErrorType) | Context manager to test for exceptions | The test function is called within this context manager
@pytest.mark.xfail(expectedfailreason) | Mark decorated fn as Expected failure | @pytest.mark.xfail(reason='Using TDD, fn not implemented')
@pytest.mark.skipif(boolean_expr, reasonskipped) | Mark decorated so pytest skips fn on given condition | @pytest.mark.skipif(sys.version_info > (2, 7), reason='requires Python 2.7')


## Test for exceptions instead of return values
- `pytest.raises(ErrorType)` uses a context manager
- General structure
	```python
	with pytest.raises(ValueError):   
		# Do nothing on entering context  
		print('This is part of the context')  
		# if code in this context raises ValueError, good. was expected  
		# if code in this context did not raise ValueEror, raise an exception  
	```

In [31]:
### Test for exception example
def add(a, b):
    if not isinstance(a, int) or not isinstance(b, int):
        raise TypeError('Must pass two integers :)')
    return a+b

# GOOD
def test_typeerror_on_non_int():
    #Test to make sure Error is raised
    with pytest.raises(TypeError) as exception_info:  
        # Doesn't do anything if context raised TypeError
        add('3', 5)
    # Test for correct message
    assert exception_info.match('Must pass two integers :)')

# WILL FAIL
def test_typeerror_on_non_int():
    #Test to make sure Error is raised
    with pytest.raises(TypeError) as exception_info:  
        print('No code in this context will raise TypeError so will fail')
    # Test for correct message
    assert exception_info.match('Must pass two integers :)')

## Test Setup and Teardown

### Fixtures
- Fixures run setup code then `yield` or `return` some data/conn/etc then optionally run tear down code
- Examples of fixtures:
	-	Some functions may need access to new blank file in some directory

#### Create a Fixture
- Create with `@pytest.fixture`

In [32]:
import os
@pytest.fixture
def new_blank_file():
    file = open('new_blank_file.txt', 'w')  # setup
    yield file
    os.remove(file)  # teardown

#### Use Fixture
- Use fixture by passing into test function as parameter

In [33]:
# Actual fn being tested
def fn_requires_blnk_file(new_file):
    new_file.addedattr = 'Hello new file'
    return new_file

# Pass fixture in as an argument
def test_on_fn_that_requires_new_blank_file(new_blank_file):
    actual = fn_requires_blnk_file(new_blank_file).addedattr
    expected = 'Hello new file'
    assert expected == actual, f'fn_requires_blnk_file(new_blank_file).addedattr should return {expected} but got {actual}'

#### Use Multiple Fixtures - Fixture chaining
- Use fixture in another fixture


In [34]:
# tmpdir is a pytest built-in fixture which setups up tmp dir and tears it down afterwards
@pytest.fixture
def new_blank_file(tmpdir):
    # create new blank file in a temp directory
    file_in_tmp_dir = tmpdir.join('new_blank_file.txt')
    open(file_in_tmp_dir, 'w').close()
    yield file_in_tmp_dir
    # teardown not needed b/c tmpdir has tear down code

## Mocking
- Test functions independently of their dependencies
- Replace a dependency with a bug-free fake (mock)
- Use `pytest-mock`
	- *Preferred when using pytest*
	- Undoes the mocking automatically after the end of the test
	- Provides other nice utilities such as spy and stub
	- Uses pytest introspection when comparing calls
- Use `unittest.mock.MagicMock`
	- MagicMock includes magic methods
	- alternative mocker

In [35]:
# pytest_mock.mocker.patch() example

# preprocess() takes a raw file and clean file path.  
#   Inside preprocess, it runs two functions on the raw file 
#   then writes the clean data to clean file.
#   The two dependencies are: row_to_list() and convert_to_int()
# I want to test preprocess indenpendently of the two dependecies
# Two bug_free mock functions will replace the real dependencies

# These functions in: data.preprocessing_helpers directory
def row_to_list(row_str):
    # buggy fn
    return row_str.split('addbug')

def convert_to_int(num_str):
    # buggy fn
    return int(num_str, 'addbug')

def preprocess(raw, clean):
    raw_read = open(raw)
    semi_cleaned = row_to_list(raw_read)   # dependency
    cleaned = convert_to_int(semi_cleaned) # dependency
    open(clean, 'w').write(cleaned)

# Test functions
from unittest.mock import call
from pytest_mock import mocker
# Define two bug free versions of dependencies.
# Now if the test fails, it's b/c bug in preprocess not bug in dependency
def row_to_list_bug_free(row_str):
    return_values = {"1,801\t201,411\n": ["1,801", "201,411"],
                     "1,767565,112\n": None,
                     "2,002\t333,209\n": ["2,002", "333,209"],
                     "1990\t782,911\n": ["1990", "782,911"],
                     "1,285\t389129\n": ["1,285", "389129"]}
    return return_values[row_str]

def convert_to_int_bug_free(num_str):
    return_values = {"1,801": 1801, "201,411": 201411, "2,002": 2002, "333,209": 333209, "1990": None, "782,911": 782911, "1,285": 1285, "389129": None}
    return return_values[num_str]
#-------------------------------------------------------------
# Finally the actual test function
def test_on_raw_data(raw_and_clean_data_file, mocker):
    raw_path, clean_path = raw_and_clean_data_file  # fixture produces file paths
    # Replace the dependency with the bug-free mock
    # First arg is name of the path to real function
    # The side effect will be called to get faked bug-free values
    row_to_list_mock = mocker.patch('data.preprocessing_helpers.row_to_list')
    row_to_list_mock.side_effect = row_to_list_bug_free  # alt way to add side_effect
    # Replace the dependency with the bug-free mock
    # First arg is name of the path to real function
    # The side effect will be called to get faked bug-free values
    convert_to_int_mock = mocker.patch('data.preprocessing_helpers.convert_to_int', 
                                       side_effect=convert_to_int_bug_free) 
    preprocess(raw_path, clean_path)
    # How to tell if the bug_free versions were called instead of the actual fns?
    # Check if preprocess() called the dependencies correctly by checking call_args_list
    assert row_to_list_mock.call_args_list == [call("1,801\t201,411\n"), call("1,767565,112\n"), call("2,002\t333,209\n"), call("1990\t782,911\n"), call("1,285\t389129\n") ]

    assert convert_to_int_mock.call_args_list == [call("1,801"), call("201,411"), call("2,002"), call("333,209"), call("1990"), call("782,911"), call("1,285"), call("389129")]

    # Do additional checks on the clean file data
    with open(clean_path, "r") as f:
        lines = f.readlines()
    first_line = lines[0]
    assert first_line == "1801\\t201411\\n"
    second_line = lines[1]
    assert second_line == "2002\\t333209\\n" 

## Testing Data Science Models
- Difficult b/c expected return values are difficult to manually compute
- Often utilizes `r-squared` which indicates how well the model performs on unseen data
- 0 <=`r-squared`<=1
- `r-squared` == 1 indicates perfect fit

	Types of data science models | Possible solution
	--- | ---
	Linear regression | Use an exact line
	Random forest | Use inequalities and equalities<br>Set Expected `r-squared`== 0
	Support Vector Machines | Use inequalities and equalities<br>Set Expected `r-squared`== 0
	Neural Networks | Use inequalities and equalities<br>Set Expected `r-squared`== 0

## Testing plots [(DataCamp)](https://campus.datacamp.com/courses/unit-testing-for-data-science-in-python/testing-models-plots-and-much-more?ex=11)
- Test plotting functions that return matplotlib figure(s)
- Use `pytest-mpl`
	- Ignores OS related differences
	- Makes it easy to generate baseline images to test against
	- Produces a layered image to show where the differences are, if any  

1. Generate one-time baseline plot
	- Decide on test arguments which is a custom collection of mpl attributes
	- Call plotting function on test arguments
	- Convert `Figure()` to PNG image
	- Image looks OK?
	- If yes, sotre images as baseline
	- If no, fix plotting function
	
2. Test against the baseline plot
	- Call plotting function on test arguments
	- Convert `Figure()` to PNG image

## What to test  ( TODO: comeback to this )
- Should test 1-2 versions of each category
- Not all functions have bad or special arguments
- Test categories
	- Bad
		- See `Testing for exceptions` section
	- Special
		- Boundary values
		- Argument values
	- Normal

## Pytest Report
- A period indicates test passed
- 'F' indicates test failed
	- typically due to raised error
	- line containing 'where' displays actual return values

## Continuous Integration and Codecov
- Look into CircleCI
- [TravisCI](./travisci.ipynb) with Github integration

# Code Coverage
- Codecov [(via TravicCI)](./travisci.ipynb) avail with Github integration
- [Code Climate](https://codeclimate.com/quality)
	- Automated code review comments on pull requests
	- Never merge code without sufficient tests
	- Identify hot sppots to focus on what matters